Esta notebook toma audio del microfono y realiza predicciones en tiempo real 

In [1]:
import os

# Set current working directory to parent folder
os.chdir(os.path.abspath(".."))

In [38]:
import numpy as np
import pandas as pd
import pyaudio
import time
import librosa
import pickle
from src.metricas import classical_features

import keras
from keras.models import load_model

In [3]:
with open('./data/MODELS/scaler','rb') as f:
    scaler = pickle.load(f)

with open('./data/MODELS/encoder','rb') as f:
    encoder = pickle.load(f)

In [36]:
model = load_model('./data/MODELS/model.keras')

In [48]:
class AudioHandler(object):
    def __init__(self):
        self.FORMAT = pyaudio.paFloat32
        self.CHANNELS = 1
        self.RATE = 44100
        self.CHUNK = 1024 * 2
        self.p = None
        self.stream = None
        self.X = []
        self.features = []

    def start(self):
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(format=self.FORMAT,
                                  channels=self.CHANNELS,
                                  rate=self.RATE,
                                  input=True,
                                  output=False,
                                  stream_callback = self.callback,
                                  frames_per_buffer = self.CHUNK)

    def stop(self):
        self.stream.close()
        self.p.terminate()

    def callback(self, in_data, frame_count, time_info, flag):
        numpy_array = np.frombuffer(in_data, dtype=np.float32)
        self.X.append(numpy_array)
        return None, pyaudio.paContinue

    def mainloop(self):
        i = 0
        while i < 2:
            time.sleep(2.5)
            f = classical_features(np.array(self.X).ravel(), sample_rate = self.RATE)
            
            X = scaler.transform(f.reshape(1, -1))
            X = np.expand_dims(X, axis=2)
            
            pred_test = model.predict(X)
            y_pred = encoder.inverse_transform(pred_test)
            print(y_pred.flatten())
            self.X = []
            i += 1


audio = AudioHandler()
audio.start()     # open the the stream
audio.mainloop()  # main operations with librosa
audio.stop()

1/1 [==============================] - 0s 17ms/step
['sadness']
1/1 [==============================] - 0s 16ms/step
['sadness']
